In [1]:
cd ~/LoFTR/

/home/shuhei.yokoo/LoFTR


In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sns.set_style('white')

pd.set_option('precision', 5)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 100)
pd.set_option('mode.chained_assignment', None)

%load_ext autoreload
%autoreload 2

import plotly.express as px
from plotly.offline import init_notebook_mode
init_notebook_mode(True)

import torch
import cv2
import numpy as np
import matplotlib.cm as cm
from src.utils.plotting import make_matching_figure
from src.loftr import LoFTR, default_cfg

/home/shuhei.yokoo/anaconda3/envs/fbisc/lib/python3.7/site-packages/kornia/augmentation/augmentation.py:1875: DeprecationWarning:

GaussianBlur is no longer maintained and will be removed from the future versions. Please use RandomGaussianBlur instead.



In [3]:
sub = pd.read_csv('/home/shuhei.yokoo/fbisc/exp/v31/extract/fb-isc-submission.csv')
gt = pd.read_csv('/home/shuhei.yokoo/fbisc/input/public_ground_truth.csv')

In [4]:
# The default config uses dual-softmax.
# The outdoor and indoor models share the same config.
# You can change the default values like thr and coarse_match_type.
matcher = LoFTR(config=default_cfg)
matcher.load_state_dict(torch.load("weights/indoor_ds.ckpt")['state_dict'], strict=False)
matcher = matcher.eval().cuda()
matcher = torch.nn.DataParallel(matcher)

In [15]:
# qid, rid = gt[gt['reference_id'].notna()].iloc[4]
qid, rid = sub[['query_id', 'reference_id']].iloc[0]

img0_pth = f'/mnt/sdb/yokoo/fbisc/input/query_images/{qid}.jpg'
img1_pth = f'/mnt/sdb/yokoo/fbisc/input/reference_images/{rid}.jpg'
img0_raw = cv2.imread(img0_pth, cv2.IMREAD_GRAYSCALE)
img1_raw = cv2.imread(img1_pth, cv2.IMREAD_GRAYSCALE)
img0_raw = cv2.resize(img0_raw, (640, 480))
img1_raw = cv2.resize(img1_raw, (640, 480))

img0 = torch.from_numpy(img0_raw)[None][None].cuda() / 255.
img1 = torch.from_numpy(img1_raw)[None][None].cuda() / 255.
batch = {'image0': img0, 'image1': img1}

# Inference with LoFTR and get prediction
with torch.no_grad():
    matcher(batch)
    mkpts0 = batch['mkpts0_f'].cpu().numpy()
    mkpts1 = batch['mkpts1_f'].cpu().numpy()
    mconf = batch['mconf'].cpu().numpy()

KeyError: 'mkpts0_f'

In [16]:
# Draw
color = cm.jet(mconf)
text = [
    'LoFTR',
    'Matches: {}'.format(len(mkpts0)),
]
fig = make_matching_figure(img0_raw, img1_raw, mkpts0, mkpts1, color, text=text)

NameError: name 'mconf' is not defined

In [5]:
class PairDataset(torch.utils.data.Dataset):
    def __init__(
        self,
        query_ids,
        reference_ids,
    ):
        self.query_ids = query_ids
        self.reference_ids = reference_ids

    def __len__(self):
        return len(self.reference_ids)

    def __getitem__(self, i):
        qid = self.query_ids[i]
        rid = self.reference_ids[i]
        img0_pth = f'/mnt/sdb/yokoo/fbisc/input/query_images/{qid}.jpg'
        img1_pth = f'/mnt/sdb/yokoo/fbisc/input/reference_images/{rid}.jpg'
        img0_raw = cv2.imread(img0_pth, cv2.IMREAD_GRAYSCALE)
        img1_raw = cv2.imread(img1_pth, cv2.IMREAD_GRAYSCALE)
        img0_raw = cv2.resize(img0_raw, (640, 480))
        img1_raw = cv2.resize(img1_raw, (640, 480))
        img0 = torch.from_numpy(img0_raw)[None] / 255.
        img1 = torch.from_numpy(img1_raw)[None] / 255.
        return img0, img1

In [6]:
# sub = sub.sort_values('score', ascending=False)

dataset = PairDataset(sub['query_id'].values, sub['reference_id'].values)
loader = torch.utils.data.DataLoader(
    dataset, batch_size=32, shuffle=False, num_workers=8, pin_memory=True, drop_last=False)

In [7]:
num_matches = []

for img0, img1 in tqdm(loader):
    img0 = img0.to('cuda')
    img1 = img1.to('cuda')

    batch = {'image0': img0, 'image1': img1}

    with torch.no_grad():
        output = matcher(batch)
        num_matches.extend(output.cpu().numpy().tolist())
#         num_matches.extend(np.bincount(output.cpu().numpy()).tolist())

  0%|          | 0/7813 [00:00<?, ?it/s]

/home/shuhei.yokoo/anaconda3/envs/fbisc/lib/python3.7/site-packages/torch/_tensor.py:575: UserWarning:

floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448265233/work/aten/src/ATen/native/BinaryOps.cpp:467.)



TypeError: zip argument #1 must support iteration

In [24]:
sub_ = sub.copy()
sub_.loc[sub_.index[:len(num_matches)], 'num_matches'] = num_matches

sub_.loc[sub_['num_matches'].notna(), 'score'] += np.log1p(sub_.loc[sub_['num_matches'].notna(), 'num_matches']) / 10
sub_.iloc[:, :3].to_csv('/home/shuhei.yokoo/fbisc/notebooks/tmp.csv', index=False)